# Number of Comments Analysis
- 1) use Post Data
- 2) Get pre-computed cluster information
- 3) Assign each question or answer to a specific cluster
- 4) Daily Aggregation by each cluster

### 1) Data Preparation

In [12]:
# Post Data
import pandas as pd
import numpy as np
import sqlite3
import pickle
import math
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm

# Import Dataset
conn = sqlite3.connect('/data1/StackOverflow/stackexchange-to-sqlite/stack.db')
query = '''
SELECT *
FROM posts
WHERE creation_date > '2021-09-01'
AND creation_date < '2023-09-01';
'''
df = pd.read_sql_query(query, conn)
conn.close()

In [14]:
df

,id,post_type,score,views,answers,comments,favorites,creation_date,closed_date,accepted_answer_id,parent_id,owner_user_id,community_owned_date,tags,title,body,last_editor_user_id,last_edit_date,last_activity_date
0,69006420,answer,0,0,0,1,0,2021-09-01 00:00:18.070,None,NaN,69006229.0,7332046.0,None,None,None,"<p>The short approach, remove python3.9 from y...",NaN,None,2021-09-01 00:00:18.070
1,69006421,answer,1,0,0,0,0,2021-09-01 00:00:28.823,None,NaN,68746577.0,14471093.0,None,None,None,"<p>open the terminal, run as administrator.</p...",3025856.0,2021-09-01 01:01:29.480,2021-09-01 01:01:29.480
2,69006422,answer,1,0,0,2,0,2021-09-01 00:00:35.537,None,NaN,69006293.0,523612.0,None,None,None,"<p>Conceptually, <code>Base</code> has no reas...",NaN,None,2021-09-01 00:00:35.537
3,69006423,question,0,184,1,0,0,2021-09-01 00:00:35.237,None,NaN,NaN,14087917.0,None,"[""node.js"", ""reactjs"", ""express"", ""axios"", ""re...",Axios Interceptors Prevent Error logging into ...,<p>Using interceptors for the first time to re...,NaN,None,2021-09-01 01:02:45.057
4,69006426,question,0,156,1,0,0,2021-09-01 00:00:55.583,None,69006735.0,NaN,13091928.0,None,"[""r"", ""shiny""]",NavbarPage to dashboardPage in a Shiny app,"<p>The APP below uses <code>navbarPage</code>,...",NaN,None,2021-09-01 04:19:58.377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5844592,77019853,answer,0,0,0,0,0,2023-08-31 23:56:47.647,None,NaN,76922631.0,5103949.0,None,None,None,"<p>I had the same error, I fixed it by followi...",NaN,None,2023-08-31 23:56:47.647
5844593,77019854,question,0,15,0,2,0,2023-08-31 23:57:28.633,None,NaN,NaN,2532775.0,None,"[""angular"", ""typescript"", ""progressive-web-app...",Prevent Using Timeout and Interval with swupda...,<p>I utilized the guidelines presented in <a h...,NaN,None,2023-08-31 23:57:28.633
5844594,77019855,answer,0,0,0,0,0,2023-08-31 23:57:36.797,None,NaN,77019829.0,16844882.0,None,None,None,"<p>In your Razor component, you can inject the...",NaN,None,2023-08-31 23:57:36.797
5844595,77019856,question,0,9,0,0,0,2023-08-31 23:58:56.693,None,NaN,NaN,2604570.0,None,"[""next.js""]",How to set the metadata of individual parallel...,"<p>I have the following structure, i am using ...",NaN,None,2023-08-31 23:58:56.693


In [23]:
df_post = df[['id', 'post_type', 'creation_date', 'answers', 'comments', 'parent_id', 'tags', 'body']]
df_post = df_post[(df_post['post_type']=='question') | (df_post['post_type'] == 'answer')]
df_post

,id,post_type,creation_date,answers,comments,parent_id,tags,body
0,69006420,answer,2021-09-01 00:00:18.070,0,1,69006229.0,None,"<p>The short approach, remove python3.9 from y..."
1,69006421,answer,2021-09-01 00:00:28.823,0,0,68746577.0,None,"<p>open the terminal, run as administrator.</p..."
2,69006422,answer,2021-09-01 00:00:35.537,0,2,69006293.0,None,"<p>Conceptually, <code>Base</code> has no reas..."
3,69006423,question,2021-09-01 00:00:35.237,1,0,NaN,"[""node.js"", ""reactjs"", ""express"", ""axios"", ""re...",<p>Using interceptors for the first time to re...
4,69006426,question,2021-09-01 00:00:55.583,1,0,NaN,"[""r"", ""shiny""]","<p>The APP below uses <code>navbarPage</code>,..."
...,...,...,...,...,...,...,...,...
5844592,77019853,answer,2023-08-31 23:56:47.647,0,0,76922631.0,None,"<p>I had the same error, I fixed it by followi..."
5844593,77019854,question,2023-08-31 23:57:28.633,0,2,NaN,"[""angular"", ""typescript"", ""progressive-web-app...",<p>I utilized the guidelines presented in <a h...
5844594,77019855,answer,2023-08-31 23:57:36.797,0,0,77019829.0,None,"<p>In your Razor component, you can inject the..."
5844595,77019856,question,2023-08-31 23:58:56.693,0,0,NaN,"[""next.js""]","<p>I have the following structure, i am using ..."


In [29]:
df_post

,id,post_type,creation_date,answers,comments,parent_id,tags,body
0,69006420,answer,2021-09-01 00:00:18.070,0,1,69006229.0,None,"<p>The short approach, remove python3.9 from y..."
1,69006421,answer,2021-09-01 00:00:28.823,0,0,68746577.0,None,"<p>open the terminal, run as administrator.</p..."
2,69006422,answer,2021-09-01 00:00:35.537,0,2,69006293.0,None,"<p>Conceptually, <code>Base</code> has no reas..."
3,69006423,question,2021-09-01 00:00:35.237,1,0,NaN,"[""node.js"", ""reactjs"", ""express"", ""axios"", ""re...",<p>Using interceptors for the first time to re...
4,69006426,question,2021-09-01 00:00:55.583,1,0,NaN,"[""r"", ""shiny""]","<p>The APP below uses <code>navbarPage</code>,..."
...,...,...,...,...,...,...,...,...
5844592,77019853,answer,2023-08-31 23:56:47.647,0,0,76922631.0,None,"<p>I had the same error, I fixed it by followi..."
5844593,77019854,question,2023-08-31 23:57:28.633,0,2,NaN,"[""angular"", ""typescript"", ""progressive-web-app...",<p>I utilized the guidelines presented in <a h...
5844594,77019855,answer,2023-08-31 23:57:36.797,0,0,77019829.0,None,"<p>In your Razor component, you can inject the..."
5844595,77019856,question,2023-08-31 23:58:56.693,0,0,NaN,"[""next.js""]","<p>I have the following structure, i am using ..."


In [30]:
# For answers data, convert the id to the parent_id.
df_post['id'] = df_post.apply(lambda row: row['parent_id'] if row['post_type'] == 'answer' else row['id'], axis = 1)
df_post

,id,post_type,creation_date,answers,comments,parent_id,tags,body
0,69006229.0,answer,2021-09-01 00:00:18.070,0,1,69006229.0,None,"<p>The short approach, remove python3.9 from y..."
1,68746577.0,answer,2021-09-01 00:00:28.823,0,0,68746577.0,None,"<p>open the terminal, run as administrator.</p..."
2,69006293.0,answer,2021-09-01 00:00:35.537,0,2,69006293.0,None,"<p>Conceptually, <code>Base</code> has no reas..."
3,69006423.0,question,2021-09-01 00:00:35.237,1,0,NaN,"[""node.js"", ""reactjs"", ""express"", ""axios"", ""re...",<p>Using interceptors for the first time to re...
4,69006426.0,question,2021-09-01 00:00:55.583,1,0,NaN,"[""r"", ""shiny""]","<p>The APP below uses <code>navbarPage</code>,..."
...,...,...,...,...,...,...,...,...
5844592,76922631.0,answer,2023-08-31 23:56:47.647,0,0,76922631.0,None,"<p>I had the same error, I fixed it by followi..."
5844593,77019854.0,question,2023-08-31 23:57:28.633,0,2,NaN,"[""angular"", ""typescript"", ""progressive-web-app...",<p>I utilized the guidelines presented in <a h...
5844594,77019829.0,answer,2023-08-31 23:57:36.797,0,0,77019829.0,None,"<p>In your Razor component, you can inject the..."
5844595,77019856.0,question,2023-08-31 23:58:56.693,0,0,NaN,"[""next.js""]","<p>I have the following structure, i am using ..."


### 2) Get Pre-computed Cluster info and merge

In [31]:
# Load pre-computed cluster info (preChatGPT).
with open(file = 'ques_df_pre.pickle', mode = 'rb') as file:
    ques_df = pickle.load(file)

In [32]:
ques_df

,id,creation_date,body,tags,community,full_body,code_body
0,69006423,2021-09-01 00:00:35.237,<p>Using interceptors for the first time to re...,"[nodeDOTjs, reactjs, express, axios, refreshto...",0,Using interceptors for the first time to refre...,
1,69006426,2021-09-01 00:00:55.583,"<p>The APP below uses <code>navbarPage</code>,...","[r, shiny]",17,"The APP below uses navbarPage, however I will ...",dashboardPage\nnavbarPage\nlibrary(shiny)\nlib...
2,69006431,2021-09-01 00:01:23.670,<p>I am developing a java project and every th...,"[java, installation, compilation, project, ope...",3,I am developing a java project and every thing...,
3,69006437,2021-09-01 00:03:18.750,"<p>As you know, all firebase hosting is provid...","[firebase, nuxtDOTjs, firebasehosting]",5,"As you know, all firebase hosting is provided ...","[\n '@nuxtjs/firebase',\n {\n ..."
4,69006439,2021-09-01 00:03:36.930,<p>I am trying to split the string as below</p...,"[r, dataDOTtable]",17,I am trying to split the string as below\nx <-...,"x <- data.table(VAROLD=c('DBA','ADB'))\nx[, pa..."
...,...,...,...,...,...,...,...
2713807,77019848,2023-08-31 23:54:30.057,<p>I need help solving a CORS issue.</p>\n<p>I...,"[docker, dockercompose, owaspdependencycheck, ...",7,I need help solving a CORS issue.\nI am trying...,- API_BASE_URL=http://XXX.XXX.XX.XX:8081\nAcce...
2713808,77019849,2023-08-31 23:55:21.660,<p>I am currently attempting to search a Share...,"[microsoftgraphapi, sharepointonline]",1,I am currently attempting to search a SharePoi...,"{\n ""requests"": [\n {\n ..."
2713809,77019852,2023-08-31 23:56:42.653,"<p>Similar to <a href=""https://stackoverflow.c...","[python, numpy, floatingpoint]",17,Similar to Numpy astype rounding to wrong valu...,In [2]: import numpy as np\n\n...\n\nIn [49]: ...
2713810,77019854,2023-08-31 23:57:28.633,<p>I utilized the guidelines presented in <a h...,"[angular, typescript, progressivewebapps, ngsw...",0,I utilized the guidelines presented in this ar...,import {BrowserModule} from '@angular/platform...


In [36]:
df_post['id'] = df_post['id'].astype(int)
df_merge = pd.merge(df_post, ques_df[['id', 'community']], on = 'id', how = 'left')
df_merge

,id,post_type,creation_date,answers,comments,parent_id,tags,body,community
0,69006229,answer,2021-09-01 00:00:18.070,0,1,69006229.0,None,"<p>The short approach, remove python3.9 from y...",NaN
1,68746577,answer,2021-09-01 00:00:28.823,0,0,68746577.0,None,"<p>open the terminal, run as administrator.</p...",NaN
2,69006293,answer,2021-09-01 00:00:35.537,0,2,69006293.0,None,"<p>Conceptually, <code>Base</code> has no reas...",NaN
3,69006423,question,2021-09-01 00:00:35.237,1,0,NaN,"[""node.js"", ""reactjs"", ""express"", ""axios"", ""re...",<p>Using interceptors for the first time to re...,0.0
4,69006426,question,2021-09-01 00:00:55.583,1,0,NaN,"[""r"", ""shiny""]","<p>The APP below uses <code>navbarPage</code>,...",17.0
...,...,...,...,...,...,...,...,...,...
5840137,76922631,answer,2023-08-31 23:56:47.647,0,0,76922631.0,None,"<p>I had the same error, I fixed it by followi...",5.0
5840138,77019854,question,2023-08-31 23:57:28.633,0,2,NaN,"[""angular"", ""typescript"", ""progressive-web-app...",<p>I utilized the guidelines presented in <a h...,0.0
5840139,77019829,answer,2023-08-31 23:57:36.797,0,0,77019829.0,None,"<p>In your Razor component, you can inject the...",1.0
5840140,77019856,question,2023-08-31 23:58:56.693,0,0,NaN,"[""next.js""]","<p>I have the following structure, i am using ...",0.0


In [40]:
df_filtered = df_merge.dropna(subset = ['community'])
df_filtered['community'] = df_filtered['community'].astype(int)
df_filtered = df_filtered[(df_filtered['community'] != 2) & (df_filtered['community'] != 12) & (df_filtered['community'] != 16)]
df_filtered

/tmp/ipykernel_2199805/1645515878.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['community'] = df_filtered['community'].astype(int)


,id,post_type,creation_date,answers,comments,parent_id,tags,body,community
3,69006423,question,2021-09-01 00:00:35.237,1,0,NaN,"[""node.js"", ""reactjs"", ""express"", ""axios"", ""re...",<p>Using interceptors for the first time to re...,0
4,69006426,question,2021-09-01 00:00:55.583,1,0,NaN,"[""r"", ""shiny""]","<p>The APP below uses <code>navbarPage</code>,...",17
7,69006431,question,2021-09-01 00:01:23.670,1,2,NaN,"[""java"", ""installation"", ""compilation"", ""proje...",<p>I am developing a java project and every th...,3
11,69006437,question,2021-09-01 00:03:18.750,1,0,NaN,"[""firebase"", ""nuxt.js"", ""firebase-hosting""]","<p>As you know, all firebase hosting is provid...",5
13,69006439,question,2021-09-01 00:03:36.930,1,2,NaN,"[""r"", ""data.table""]",<p>I am trying to split the string as below</p...,17
...,...,...,...,...,...,...,...,...,...
5840137,76922631,answer,2023-08-31 23:56:47.647,0,0,76922631.0,None,"<p>I had the same error, I fixed it by followi...",5
5840138,77019854,question,2023-08-31 23:57:28.633,0,2,NaN,"[""angular"", ""typescript"", ""progressive-web-app...",<p>I utilized the guidelines presented in <a h...,0
5840139,77019829,answer,2023-08-31 23:57:36.797,0,0,77019829.0,None,"<p>In your Razor component, you can inject the...",1
5840140,77019856,question,2023-08-31 23:58:56.693,0,0,NaN,"[""next.js""]","<p>I have the following structure, i am using ...",0


In [42]:
df_filtered.community.nunique() # 15 clusters confirmed.

15

### 3) Monthly Aggregation by each Cluster

In [44]:
#create daily timeseries data
df_filtered['year_month_day'] = pd.to_datetime(df_filtered['creation_date']).dt.to_period('D') # daily
df_filtered['year_month_day'] = df_filtered['year_month_day'].astype(str)

In [46]:
df_filtered[['year_month_day', 'community', 'comments']]

,year_month_day,community,comments
3,2021-09-01,0,0
4,2021-09-01,17,0
7,2021-09-01,3,2
11,2021-09-01,5,0
13,2021-09-01,17,2
...,...,...,...
5840137,2023-08-31,5,0
5840138,2023-08-31,0,2
5840139,2023-08-31,1,0
5840140,2023-08-31,0,0


In [47]:
numComments = df_filtered.groupby(['year_month_day', 'community'])['comments'].sum().reset_index()
numComments

,year_month_day,community,comments
0,2021-09-01,0,3620
1,2021-09-01,1,1297
2,2021-09-01,3,1133
3,2021-09-01,4,1905
4,2021-09-01,5,970
...,...,...,...
10889,2023-08-31,11,30
10890,2023-08-31,13,49
10891,2023-08-31,14,1
10892,2023-08-31,15,7


In [108]:
# Make blank matrix with dates and community info.
df_blank = pd.DataFrame({'year_month_day': np.repeat(np.array(df_filtered['year_month_day'].unique()), 15), # Duplicate each date value (730) by the number of cluster (15)
              'community': np.tile(np.array(numComments['community'].unique()), 730)}) # Duplicate cluster info 730 times(number of unique dates)
df_final = pd.merge(df_blank, numComments, on = ['year_month_day', 'community'], how = 'left')
# 730*15 observation
df_final

,year_month_day,community,comments
0,2021-09-01,0,3620.0
1,2021-09-01,1,1297.0
2,2021-09-01,3,1133.0
3,2021-09-01,4,1905.0
4,2021-09-01,5,970.0
...,...,...,...
10945,2023-08-31,11,30.0
10946,2023-08-31,13,49.0
10947,2023-08-31,14,1.0
10948,2023-08-31,15,7.0


In [109]:
# Fill NAs with 0.
df_final['comments'] = df_final['comments'].fillna(0).astype(int)

In [110]:
df_final

,year_month_day,community,comments
0,2021-09-01,0,3620
1,2021-09-01,1,1297
2,2021-09-01,3,1133
3,2021-09-01,4,1905
4,2021-09-01,5,970
...,...,...,...
10945,2023-08-31,11,30
10946,2023-08-31,13,49
10947,2023-08-31,14,1
10948,2023-08-31,15,7


# Average Number of Comments per Post

In [111]:
ComPerQ = df_filtered.groupby(['year_month_day', 'community'])['comments'].mean().reset_index(name = 'CommentsPerQ')
ComPerQ

,year_month_day,community,CommentsPerQ
0,2021-09-01,0,1.614630
1,2021-09-01,1,2.071885
2,2021-09-01,3,1.759317
3,2021-09-01,4,2.830609
4,2021-09-01,5,1.525157
...,...,...,...
10889,2023-08-31,11,1.000000
10890,2023-08-31,13,1.042553
10891,2023-08-31,14,0.111111
10892,2023-08-31,15,0.777778


In [112]:
# Merge to the final data.
df_final = pd.merge(df_final, ComPerQ, on = ['year_month_day', 'community'], how = 'left')
# Fill NAs with 0.
df_final['CommentsPerQ'] = df_final['CommentsPerQ'].fillna(0)
df_final

,year_month_day,community,comments,CommentsPerQ
0,2021-09-01,0,3620,1.614630
1,2021-09-01,1,1297,2.071885
2,2021-09-01,3,1133,1.759317
3,2021-09-01,4,1905,2.830609
4,2021-09-01,5,970,1.525157
...,...,...,...,...
10945,2023-08-31,11,30,1.000000
10946,2023-08-31,13,49,1.042553
10947,2023-08-31,14,1,0.111111
10948,2023-08-31,15,7,0.777778


In [115]:
questions = df_filtered[df_filtered['post_type'] == 'question']
AnsPerQ = questions.groupby(['year_month_day', 'community'])['answers'].mean().reset_index(name = 'AnsPerQ')
AnsPerQ

,year_month_day,community,AnsPerQ
0,2021-09-01,0,1.164056
1,2021-09-01,1,1.032419
2,2021-09-01,3,1.010152
3,2021-09-01,4,1.141304
4,2021-09-01,5,1.190476
...,...,...,...
10825,2023-08-31,11,0.666667
10826,2023-08-31,13,0.413793
10827,2023-08-31,14,0.000000
10828,2023-08-31,15,0.600000


In [116]:
# Merge to the final data.
df_final = pd.merge(df_final, AnsPerQ, on = ['year_month_day', 'community'], how = 'left')
# Fill NAs with 0.
df_final['AnsPerQ'] = df_final['AnsPerQ'].fillna(0)
df_final

,year_month_day,community,comments,CommentsPerQ,AnsPerQ
0,2021-09-01,0,3620,1.614630,1.164056
1,2021-09-01,1,1297,2.071885,1.032419
2,2021-09-01,3,1133,1.759317,1.010152
3,2021-09-01,4,1905,2.830609,1.141304
4,2021-09-01,5,970,1.525157,1.190476
...,...,...,...,...,...
10945,2023-08-31,11,30,1.000000,0.666667
10946,2023-08-31,13,49,1.042553,0.413793
10947,2023-08-31,14,1,0.111111,0.000000
10948,2023-08-31,15,7,0.777778,0.600000


In [117]:
df_final.isna().sum()

year_month_day    0
community         0
comments          0
CommentsPerQ      0
AnsPerQ           0
dtype: int64

# Final Setup for DDD and Save Data
- Original Num Observation = 730(days) * 15 (clusters) = 10,950
- As some values in count_q, count_a, or entropy equals to 0, their log values cannot be calculated.
- Therefore, after removing those observations = 10,950 - 238 = 10,712

In [118]:
df_final_pre4 = pd.read_csv("df_final_pre4.csv")
df_final_pre4

,year_month_day,T_d,P_t,month,community,techiness,entropy,count_q,count_a,ln_q,ln_a,ln_entropy,year_month,numUser,ln_numUser
0,2021-09-01,0,0,9,0,0.525040,7.298398,1213.0,1029.0,7.100852,6.936343,1.987655,2021-09,22317,10.013104
1,2021-09-01,0,0,9,1,0.412791,7.399966,401.0,225.0,5.993961,5.416100,2.001475,2021-09,7820,8.964440
2,2021-09-01,0,0,9,3,0.490224,7.231138,394.0,250.0,5.976351,5.521461,1.978396,2021-09,7714,8.950792
3,2021-09-01,0,0,9,4,0.439161,7.709924,368.0,305.0,5.908083,5.720312,2.042508,2021-09,7720,8.951570
4,2021-09-01,0,0,9,5,0.461350,7.043218,378.0,258.0,5.934894,5.552960,1.952065,2021-09,7509,8.923858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10707,2023-08-31,1,1,8,11,0.476285,4.175869,18.0,12.0,2.890372,2.484907,1.429322,2023-08,343,5.837730
10708,2023-08-31,1,1,8,13,0.421145,5.528088,29.0,18.0,3.367296,2.890372,1.709842,2023-08,630,6.445720
10709,2023-08-31,1,1,8,14,0.403302,3.344698,8.0,1.0,2.079442,0.000000,1.207377,2023-08,278,5.627621
10710,2023-08-31,1,1,8,15,0.307775,2.947703,5.0,4.0,1.609438,1.386294,1.081026,2023-08,122,4.804021


In [124]:
df_final_pre6 = pd.merge(df_final_pre4, df_final, on = ['year_month_day', 'community'], how = 'left')
df_final_pre6

,year_month_day,T_d,P_t,month,community,techiness,entropy,count_q,count_a,ln_q,ln_a,ln_entropy,year_month,numUser,ln_numUser,comments,CommentsPerQ,AnsPerQ
0,2021-09-01,0,0,9,0,0.525040,7.298398,1213.0,1029.0,7.100852,6.936343,1.987655,2021-09,22317,10.013104,3620,1.614630,1.164056
1,2021-09-01,0,0,9,1,0.412791,7.399966,401.0,225.0,5.993961,5.416100,2.001475,2021-09,7820,8.964440,1297,2.071885,1.032419
2,2021-09-01,0,0,9,3,0.490224,7.231138,394.0,250.0,5.976351,5.521461,1.978396,2021-09,7714,8.950792,1133,1.759317,1.010152
3,2021-09-01,0,0,9,4,0.439161,7.709924,368.0,305.0,5.908083,5.720312,2.042508,2021-09,7720,8.951570,1905,2.830609,1.141304
4,2021-09-01,0,0,9,5,0.461350,7.043218,378.0,258.0,5.934894,5.552960,1.952065,2021-09,7509,8.923858,970,1.525157,1.190476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10707,2023-08-31,1,1,8,11,0.476285,4.175869,18.0,12.0,2.890372,2.484907,1.429322,2023-08,343,5.837730,30,1.000000,0.666667
10708,2023-08-31,1,1,8,13,0.421145,5.528088,29.0,18.0,3.367296,2.890372,1.709842,2023-08,630,6.445720,49,1.042553,0.413793
10709,2023-08-31,1,1,8,14,0.403302,3.344698,8.0,1.0,2.079442,0.000000,1.207377,2023-08,278,5.627621,1,0.111111,0.000000
10710,2023-08-31,1,1,8,15,0.307775,2.947703,5.0,4.0,1.609438,1.386294,1.081026,2023-08,122,4.804021,7,0.777778,0.600000


In [135]:
# There are 55 0 values encountered in comments variable.
print("There are %d zeros encountered in Comments variable" %len(df_final_pre6[df_final_pre6['comments'] == 0]))
print("There are %d zeros encountered in AnsPerQ variable" %len(df_final_pre6[df_final_pre6['AnsPerQ'] == 0]))
print("There are %d zeros encountered in CommentsPerQ variable" %len(df_final_pre6[df_final_pre6['CommentsPerQ'] == 0]))

There are 55 zeros encountered in Comments variable
There are 33 zeros encountered in AnsPerQ variable
There are 55 zeros encountered in CommentsPerQ variable


In [138]:
# Remove rows with 0 value (10,712->10,474)
df_final_pre6 = df_final_pre6[(df_final_pre6['comments'] != 0)&(df_final_pre6['AnsPerQ'] != 0)&(df_final_pre6['CommentsPerQ'] != 0)]
df_final_pre6

,year_month_day,T_d,P_t,month,community,techiness,entropy,count_q,count_a,ln_q,ln_a,ln_entropy,year_month,numUser,ln_numUser,comments,CommentsPerQ,AnsPerQ
0,2021-09-01,0,0,9,0,0.525040,7.298398,1213.0,1029.0,7.100852,6.936343,1.987655,2021-09,22317,10.013104,3620,1.614630,1.164056
1,2021-09-01,0,0,9,1,0.412791,7.399966,401.0,225.0,5.993961,5.416100,2.001475,2021-09,7820,8.964440,1297,2.071885,1.032419
2,2021-09-01,0,0,9,3,0.490224,7.231138,394.0,250.0,5.976351,5.521461,1.978396,2021-09,7714,8.950792,1133,1.759317,1.010152
3,2021-09-01,0,0,9,4,0.439161,7.709924,368.0,305.0,5.908083,5.720312,2.042508,2021-09,7720,8.951570,1905,2.830609,1.141304
4,2021-09-01,0,0,9,5,0.461350,7.043218,378.0,258.0,5.934894,5.552960,1.952065,2021-09,7509,8.923858,970,1.525157,1.190476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10706,2023-08-31,1,1,8,10,0.373225,2.947703,3.0,1.0,1.098612,0.000000,1.081026,2023-08,75,4.317488,1,0.250000,0.333333
10707,2023-08-31,1,1,8,11,0.476285,4.175869,18.0,12.0,2.890372,2.484907,1.429322,2023-08,343,5.837730,30,1.000000,0.666667
10708,2023-08-31,1,1,8,13,0.421145,5.528088,29.0,18.0,3.367296,2.890372,1.709842,2023-08,630,6.445720,49,1.042553,0.413793
10710,2023-08-31,1,1,8,15,0.307775,2.947703,5.0,4.0,1.609438,1.386294,1.081026,2023-08,122,4.804021,7,0.777778,0.600000


In [139]:
df_final_pre6['ln_comments'] = np.log(df_final_pre6['comments'])
df_final_pre6['ln_AnsPerQ'] = np.log(df_final_pre6['AnsPerQ'])
df_final_pre6['ln_CommentsPerQ'] = np.log(df_final_pre6['CommentsPerQ'])
df_final_pre6

/tmp/ipykernel_2199805/3881698927.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_pre6['ln_comments'] = np.log(df_final_pre6['comments'])
/tmp/ipykernel_2199805/3881698927.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_pre6['ln_AnsPerQ'] = np.log(df_final_pre6['AnsPerQ'])
/tmp/ipykernel_2199805/3881698927.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

,year_month_day,T_d,P_t,month,community,techiness,entropy,count_q,count_a,ln_q,...,ln_entropy,year_month,numUser,ln_numUser,comments,CommentsPerQ,AnsPerQ,ln_comments,ln_AnsPerQ,ln_CommentsPerQ
0,2021-09-01,0,0,9,0,0.525040,7.298398,1213.0,1029.0,7.100852,...,1.987655,2021-09,22317,10.013104,3620,1.614630,1.164056,8.194229,0.151911,0.479106
1,2021-09-01,0,0,9,1,0.412791,7.399966,401.0,225.0,5.993961,...,2.001475,2021-09,7820,8.964440,1297,2.071885,1.032419,7.167809,0.031905,0.728459
2,2021-09-01,0,0,9,3,0.490224,7.231138,394.0,250.0,5.976351,...,1.978396,2021-09,7714,8.950792,1133,1.759317,1.010152,7.032624,0.010101,0.564926
3,2021-09-01,0,0,9,4,0.439161,7.709924,368.0,305.0,5.908083,...,2.042508,2021-09,7720,8.951570,1905,2.830609,1.141304,7.552237,0.132172,1.040492
4,2021-09-01,0,0,9,5,0.461350,7.043218,378.0,258.0,5.934894,...,1.952065,2021-09,7509,8.923858,970,1.525157,1.190476,6.877296,0.174353,0.422098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10706,2023-08-31,1,1,8,10,0.373225,2.947703,3.0,1.0,1.098612,...,1.081026,2023-08,75,4.317488,1,0.250000,0.333333,0.000000,-1.098612,-1.386294
10707,2023-08-31,1,1,8,11,0.476285,4.175869,18.0,12.0,2.890372,...,1.429322,2023-08,343,5.837730,30,1.000000,0.666667,3.401197,-0.405465,0.000000
10708,2023-08-31,1,1,8,13,0.421145,5.528088,29.0,18.0,3.367296,...,1.709842,2023-08,630,6.445720,49,1.042553,0.413793,3.891820,-0.882389,0.041673
10710,2023-08-31,1,1,8,15,0.307775,2.947703,5.0,4.0,1.609438,...,1.081026,2023-08,122,4.804021,7,0.777778,0.600000,1.945910,-0.510826,-0.251314


In [140]:
df_final_pre6.describe()

,T_d,P_t,month,community,techiness,entropy,count_q,count_a,ln_q,ln_a,ln_entropy,numUser,ln_numUser,comments,CommentsPerQ,AnsPerQ,ln_comments,ln_AnsPerQ,ln_CommentsPerQ
count,10630.000000,10630.000000,10630.000000,10630.000000,10630.000000,10630.000000,10630.000000,10630.000000,10630.000000,10630.000000,10630.000000,10630.000000,10630.000000,10630.000000,10630.000000,10630.000000,10630.000000,10630.000000,10630.000000
mean,0.498495,0.752117,6.528222,8.099341,0.428349,5.819072,255.253340,238.264158,4.512480,4.401888,1.703280,6159.109125,7.760997,720.714958,1.356345,0.910748,5.393026,-0.125621,0.231363
std,0.500021,0.431804,3.451245,5.007127,0.070252,1.744202,306.257494,303.363132,1.737208,1.758136,0.366922,6789.009263,1.702006,865.540759,0.462259,0.230973,1.957291,0.259518,0.422676
min,0.000000,0.000000,1.000000,0.000000,0.304784,0.918296,1.000000,1.000000,0.000000,0.000000,-0.085236,48.000000,3.871201,1.000000,0.076923,0.111111,0.000000,-2.197225,-2.564949
25%,0.000000,1.000000,4.000000,4.000000,0.403302,4.284346,22.000000,20.000000,3.091042,2.995732,1.454968,554.000000,6.317165,47.000000,1.094935,0.759073,3.850148,-0.275657,0.090695
50%,0.000000,1.000000,7.000000,8.000000,0.421145,6.618436,166.000000,137.000000,5.111988,4.919981,1.889859,5071.000000,8.531293,411.000000,1.362065,0.920064,6.018593,-0.083312,0.309002
75%,1.000000,1.000000,10.000000,13.000000,0.483382,7.335601,364.000000,332.000000,5.897154,5.805135,1.992739,8089.000000,8.998260,1104.000000,1.616703,1.053036,7.006695,0.051677,0.480389
max,1.000000,1.000000,12.000000,17.000000,0.541652,8.300774,1437.000000,1364.000000,7.270313,7.218177,2.116349,26829.000000,10.197239,4024.000000,5.333333,4.000000,8.300032,1.386294,1.673976


In [149]:
df_final_pre6 = df_final_pre6.rename(columns = {'CommentsPerQ':'CommentsPerP',
                                'ln_CommentsPerQ': 'ln_CommentsPerP'})

In [150]:
# Save Data
df_final_pre6.to_csv('df_final_pre6.csv', index=False)